In [16]:
import cv2 as cv
import numpy as np

In [ ]:
camera = cv.VideoCapture(0)
ret, frame = camera.read()

face_cascade = cv.CascadeClassifier(
    '../../data/Files/DATA/haarcascades/haarcascade_frontalface_default.xml')

# returns all the numpy arrays where it is detecting a face
face_rectangles = face_cascade.detectMultiScale(frame)


if len(face_rectangles):
    
  # detect only one face
  face_window = tuple(face_rectangles[0])

  (face_x, face_y, w, h) = face_window

  region_of_interest = frame[face_y:face_y+h, face_x:face_x+w]
  hsv_roi = cv.cvtColor(region_of_interest, cv.COLOR_BGR2HSV)

  ROI_hist = cv.calcHist([hsv_roi], [0], None, [180], [0, 180])

  cv.normalize(ROI_hist, ROI_hist, 0, 255, cv.NORM_MINMAX)


  TERMINATION_CRITERIA = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1)

  while True:
    ret, frame = camera.read()

    if ret:
      hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
      dst = cv.calcBackProject([hsv], [0], ROI_hist, [0, 180], 1)

      ###########################################
      ret, face_window = cv.CamShift(dst, face_window, TERMINATION_CRITERIA)

      new_box = cv.boxPoints(ret)
      # convert to integer
      new_box = np.int64(new_box)

      image_2 = cv.polylines(frame, [new_box],True, (0, 0, 255),5)
      ###########################################

      cv.imshow("Image", image_2)
      k = cv.waitKey(1) & 0xFF

      if k == 27:
        break

    else:
      break
else:
  print("No Face Detected")

cv.destroyAllWindows()
camera.release()